In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
%run "../../common/matrix.py"
%run "../../common/function_classifier.py"
%run "../../common/Functions.py"
%run "../../common/file_handler.py"
%run "../../common/grid_search_sgd.py"
%run "../../common/plot_functions.py"

In [ ]:
FI_XSIZE = 20
FI_YSIZE = 10

FI_ALL_XSIZE = 30
FI_ALL_YSIZE = 15

PERF_XSIZE = 10
PERF_YSIZE = 12

In [ ]:
FIGSAVE = False

In [ ]:
imported_matrix = open_file_from_fullpath(ask_file_path(), 'r')
matrix_tobe_classified = pd.read_perseus(imported_matrix)

In [ ]:
matrix_json = open_file_from_fullpath(ask_file_path(), 'r')

Drop QC

In [ ]:
matrix_tobe_cl_noqc = slice_submatrix(matrix_tobe_classified,get_value_from_json(matrix_json,'sample_slice'))
dictionary_cl_noqc, matrix_tobe_cl_noqc = rename_matrixheader_labels(matrix_tobe_cl_noqc, get_value_from_json(matrix_json,'sample_rename'))

In [ ]:
binary_label_OC = create_labels(matrix_tobe_cl_noqc, drop= ['post', 'local'])
binary_features_OC = create_feature_matrix(matrix_tobe_cl_noqc, drop=['post', 'local'])

In [ ]:
binary_label_OL = create_labels(matrix_tobe_cl_noqc, drop = ['control', 'post'])
binary_features_OL = create_feature_matrix(matrix_tobe_cl_noqc, drop=  ['control', 'post'])

In [ ]:
binary_label_CL = create_labels(matrix_tobe_cl_noqc, drop = ['post', 'onset'])
binary_features_CL = create_feature_matrix(matrix_tobe_cl_noqc, drop = ['post', 'onset'])

In [ ]:
binary_label_OvsCL = create_labels_binary(matrix_tobe_cl_noqc, 'onset' ,'control_local', ['onset'], ['control', 'local'], discard=['post'])
binary_features_OvsCL = create_feature_matrix(matrix_tobe_cl_noqc, drop= 'post')

Global params for grid search

In [ ]:
params_grid_l1 = {'alpha' : [0.0001, 0.005, 0.01, 0.05, 0.08, 0.1, 0.5, 0.7, 1.5, 1.9], 'penalty': ['l1'], 'loss': ['hinge']}
params_grid_l2 = {'alpha' : [0.0001, 0.005, 0.01, 0.05, 0.08, 0.1, 0.5, 0.7, 1.5, 1.9], 'penalty': ['l2'], 'loss': ['hinge']}
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)

Splitting data

In [ ]:
Xbin_OC_train, Xbin_OC_test, ybin_OC_train, ybin_OC_test = train_test_split(binary_features_OC, binary_label_OC, test_size= 0.20, random_state=0, stratify= binary_label_OC)
Xbin_OL_train, Xbin_OL_test, ybin_OL_train, ybin_OL_test = train_test_split(binary_features_OL, binary_label_OL, test_size= 0.20, random_state=0, stratify= binary_label_OL)
Xbin_CL_train, Xbin_CL_test, ybin_CL_train, ybin_CL_test = train_test_split(binary_features_CL, binary_label_CL, test_size= 0.20, random_state=0, stratify= binary_label_CL)
Xbin_OvsCL_train, Xbin_OvsCL_test, ybin_OvsCL_train, ybin_OvsCL_test = train_test_split(binary_features_OvsCL, binary_label_OvsCL, test_size= 0.20, random_state=0, stratify= binary_label_OvsCL)

Gridsearch

In [ ]:
binary_grid_OC_l1 = grid_search_sgd(features=Xbin_OC_train, labels=ybin_OC_train, parameters=params_grid_l1, cv=skf, name="", checkpoint_path=None, checkpoint=False)
binary_grid_OC_l2 = grid_search_sgd(features=Xbin_OC_train, labels=ybin_OC_train, parameters=params_grid_l2, cv=skf, name="", checkpoint_path=None, checkpoint=False)

binary_grid_OL_l1 = grid_search_sgd(features=Xbin_OL_train, labels=ybin_OL_train, parameters=params_grid_l1, cv=skf, name="", checkpoint_path=None, checkpoint=False)
binary_grid_OL_l2 = grid_search_sgd(features=Xbin_OL_train, labels=ybin_OL_train, parameters=params_grid_l2, cv=skf, name="", checkpoint_path=None, checkpoint=False)

binary_grid_CL_l1 = grid_search_sgd(features=Xbin_CL_train, labels=ybin_CL_train, parameters=params_grid_l1, cv=skf, name="", checkpoint_path=None, checkpoint=False)
binary_grid_CL_l2 = grid_search_sgd(features=Xbin_CL_train, labels=ybin_CL_train, parameters=params_grid_l2, cv=skf, name="", checkpoint_path=None, checkpoint=False)

binary_grid_OvsCL_l1 = grid_search_sgd(features=Xbin_OvsCL_train, labels=ybin_OvsCL_train, parameters=params_grid_l1, cv=skf, name="", checkpoint_path=None, checkpoint=False)
binary_grid_OvsCL_l2 = grid_search_sgd(features=Xbin_OvsCL_train, labels=ybin_OvsCL_train, parameters=params_grid_l2, cv=skf, name="", checkpoint_path=None, checkpoint=False)

Training classifiers

In [ ]:
binary_classifier_OC_l1 = SGDClassifier(loss='hinge', penalty='l1', alpha=binary_grid_OC_l1[0].best_params_['alpha'])
binary_classifier_OC_l1.fit(Xbin_OC_train, ybin_OC_train)
binary_classifier_OC_l2 = SGDClassifier(loss='hinge', penalty='l2', alpha=binary_grid_OC_l2[0].best_params_['alpha'])
binary_classifier_OC_l2.fit(Xbin_OC_train, ybin_OC_train)

In [ ]:
binary_classifier_OL_l1 = SGDClassifier(loss='hinge', penalty='l1', alpha=binary_grid_OL_l1[0].best_params_['alpha'])
binary_classifier_OL_l1.fit(Xbin_OL_train, ybin_OL_train)
binary_classifier_OL_l2 = SGDClassifier(loss='hinge', penalty='l2', alpha=binary_grid_OL_l2[0].best_params_['alpha'])
binary_classifier_OL_l2.fit(Xbin_OL_train, ybin_OL_train)

In [ ]:
binary_classifier_CL_l1 = SGDClassifier(loss='hinge', penalty='l1', alpha=binary_grid_CL_l1[0].best_params_['alpha'])
binary_classifier_CL_l1.fit(Xbin_CL_train, ybin_CL_train)
binary_classifier_CL_l2 = SGDClassifier(loss='hinge', penalty='l2', alpha=binary_grid_CL_l2[0].best_params_['alpha'])
binary_classifier_CL_l2.fit(Xbin_CL_train, ybin_CL_train)

In [ ]:
binary_classifier_OvsCL_l1 = SGDClassifier(loss='hinge', penalty='l1', alpha=binary_grid_OvsCL_l1[0].best_params_['alpha'])
binary_classifier_OvsCL_l1.fit(Xbin_OvsCL_train, ybin_OvsCL_train)
binary_classifier_OvsCL_l2 = SGDClassifier(loss='hinge', penalty='l2', alpha=binary_grid_OvsCL_l2[0].best_params_['alpha'])
binary_classifier_OvsCL_l2.fit(Xbin_OvsCL_train, ybin_OvsCL_train)

Values of classifiers

In [ ]:
classifiers_params = get_best_value([binary_classifier_OC_l1, binary_classifier_OC_l2, binary_classifier_OL_l1, binary_classifier_OL_l2, binary_classifier_CL_l1, binary_classifier_CL_l2, binary_classifier_OvsCL_l1, binary_classifier_OvsCL_l2])
print(classifiers_params)

In [ ]:
save_classifiers_params = open(generate_new_file_path(imported_matrix, '_sgd_classifiers_params'),'w')
save_classifiers_params.write(classifiers_params)
save_classifiers_params.close()

In [ ]:
ybinary_prediction_OC_l1 = binary_classifier_OC_l1.predict(Xbin_OC_test)
ybinary_prediction_OC_l2 = binary_classifier_OC_l2.predict(Xbin_OC_test)

ybinary_prediction_OL_l1 = binary_classifier_OL_l1.predict(Xbin_OL_test)
ybinary_prediction_OL_l2 = binary_classifier_OL_l2.predict(Xbin_OL_test)

ybinary_prediction_CL_l1 = binary_classifier_CL_l1.predict(Xbin_CL_test)
ybinary_prediction_CL_l2 = binary_classifier_CL_l2.predict(Xbin_CL_test)

ybinary_prediction_OvsCL_l1 = binary_classifier_OvsCL_l1.predict(Xbin_OvsCL_test)
ybinary_prediction_OvsCL_l2 = binary_classifier_OvsCL_l2.predict(Xbin_OvsCL_test)

In [ ]:
scores_OC_l1 = print_results(ybin_OC_test, ybinary_prediction_OC_l1)
scores_OC_l2 = print_results(ybin_OC_test, ybinary_prediction_OC_l2)

scores_OL_l1 = print_results(ybin_OL_test, ybinary_prediction_OL_l1)
scores_OL_l2 = print_results(ybin_OL_test, ybinary_prediction_OL_l2)

scores_CL_l1 = print_results(ybin_CL_test, ybinary_prediction_CL_l1)
scores_CL_l2 = print_results(ybin_CL_test, ybinary_prediction_CL_l2)

scores_OvsCL_l1 = print_results(ybin_OvsCL_test, ybinary_prediction_OvsCL_l1)
scores_OvsCL_l2 = print_results(ybin_OvsCL_test, ybinary_prediction_OvsCL_l2)

all_scores = 'onset vs control\n' + scores_OC_l1 + '\n' + scores_OC_l2 + '\nonset vs local\n' + scores_OL_l1 + '\n' + scores_OL_l2 + '\ncontrol vs local\n' + scores_CL_l1 + '\n' + scores_CL_l2 + '\nonset vs control-local\n' + scores_OvsCL_l1 + '\n' + scores_OvsCL_l2 + '\n'

In [ ]:
save_scoring = open(generate_new_file_path(imported_matrix, '_scoring_all_sgd_classifiers'), 'w')
save_scoring.write(all_scores)
save_scoring.close()

In [ ]:
binary_OC_df = create_dataframe(['X', 'Y'], [[binary_classifier_OC_l1.penalty, binary_classifier_OC_l2.penalty],
[balanced_accuracy_score(ybin_OC_test, ybinary_prediction_OC_l1), balanced_accuracy_score(ybin_OC_test, ybinary_prediction_OC_l2)]])

binary_OL_df = create_dataframe(['X', 'Y'], [[binary_classifier_OL_l1.penalty, binary_classifier_OL_l2.penalty],
[balanced_accuracy_score(ybin_OL_test, ybinary_prediction_OL_l1), balanced_accuracy_score(ybin_OL_test, ybinary_prediction_OL_l2)]])

binary_CL_df = create_dataframe(['X', 'Y'], [[binary_classifier_CL_l1.penalty, binary_classifier_CL_l2.penalty],
[balanced_accuracy_score(ybin_CL_test, ybinary_prediction_CL_l1), balanced_accuracy_score(ybin_CL_test, ybinary_prediction_CL_l2)]])

binary_OvsCL_df = create_dataframe(['X', 'Y'], [[binary_classifier_OvsCL_l1.penalty, binary_classifier_OvsCL_l2.penalty],
[balanced_accuracy_score(ybin_OvsCL_test, ybinary_prediction_OvsCL_l1), balanced_accuracy_score(ybin_OvsCL_test, ybinary_prediction_OvsCL_l2)]])

In [ ]:
display_performances_wl([binary_OC_df], ['l1', 'l2'], get_file_name(imported_matrix) + ' - SGD classifier - Performances of binary classification: Onset vs Control', 'Penalty', 'Performances', PERF_XSIZE, PERF_YSIZE, ['*'], 400, ['onset vs control'], FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_Performances_all_classificators_OC_', '.svg'))

In [ ]:
display_performances_wl([binary_OL_df], ['l1', 'l2'], get_file_name(imported_matrix) + ' - SGD classifier - Performances of binary classification: Onset vs Local', 'Penalty', 'Performances', PERF_XSIZE, PERF_YSIZE, ['*'], 400, ['onset vs local'], FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_Performances_all_classificators_OL_', '.svg'))

In [ ]:
display_performances_wl([binary_CL_df], ['l1', 'l2'], get_file_name(imported_matrix) + ' - SGD classifier - Performances of binary classification: Control vs Local', 'Penalty', 'Performances', PERF_XSIZE, PERF_YSIZE, ['*'], 400, ['control vs local'], FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_Performances_all_classificators_CL_', '.svg'))

In [ ]:
display_performances_wl([binary_OvsCL_df], ['l1', 'l2'], get_file_name(imported_matrix) + ' - SGD classifier - Performances of binary classification: Onset vs Control-Local', 'Penalty', 'Performances', PERF_XSIZE, PERF_YSIZE, ['*'], 400, ['onset vs control_local'], FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_Performances_all_classificators_OvsCL_', '.svg'))

In [ ]:
cm_binary_OC_l1 = confusion_matrix(ybin_OC_test, ybinary_prediction_OC_l1, normalize='pred')
cm_binary_OC_l2 = confusion_matrix(ybin_OC_test, ybinary_prediction_OC_l2, normalize='pred')

cm_binary_OL_l1 = confusion_matrix(ybin_OL_test, ybinary_prediction_OL_l1, normalize='pred')
cm_binary_OL_l2 = confusion_matrix(ybin_OL_test, ybinary_prediction_OL_l2, normalize='pred')

cm_binary_CL_l1 = confusion_matrix(ybin_CL_test, ybinary_prediction_CL_l1, normalize='pred')
cm_binary_CL_l2 = confusion_matrix(ybin_CL_test, ybinary_prediction_CL_l2, normalize='pred')

cm_binary_OvsCL_l1 = confusion_matrix(ybin_OvsCL_test, ybinary_prediction_OvsCL_l1, normalize='pred')
cm_binary_OvsCL_l2 = confusion_matrix(ybin_OvsCL_test, ybinary_prediction_OvsCL_l2, normalize='pred')

In [ ]:
display_confusion_matrices_binary_OC = function_subplots_confusionmatrix([cm_binary_OC_l1, cm_binary_OC_l2], 1, 2, ['binary classification l1', 'binary classification l2'], get_file_name(imported_matrix) + ' - all SGD binary classification control vs onset confusion matrices', 25, 30, labels=[binary_classifier_OC_l1.classes_, binary_classifier_OC_l2.classes_], fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_confusion_matrix_binary_OC', '.svg'))

In [ ]:
display_confusion_matrices_binary_OL = function_subplots_confusionmatrix([cm_binary_OL_l1, cm_binary_OL_l2], 1, 2, ['binary classification l1', 'binary classification l2'], get_file_name(imported_matrix) + ' - all SGD binary classification local vs onset confusion matrices', 25, 30, labels=[binary_classifier_OL_l1.classes_, binary_classifier_OL_l2.classes_], fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_confusion_matrix_binary_OL', '.svg'))

In [ ]:
display_confusion_matrices_binary_CL = function_subplots_confusionmatrix([cm_binary_CL_l1, cm_binary_CL_l2], 1, 2, ['binary classification l1', 'binary classification l2'], get_file_name(imported_matrix) + ' - all SGD binary classification local vs control confusion matrices', 25, 30, labels=[binary_classifier_CL_l1.classes_, binary_classifier_CL_l2.classes_], fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_confusion_matrix_binary_CL', '.svg'))

In [ ]:
display_confusion_matrices_OvsCL = function_subplots_confusionmatrix([cm_binary_OvsCL_l1, cm_binary_OvsCL_l2], 1, 2, ['binary classification l1', 'binary classification l2'], get_file_name(imported_matrix) + ' - all SGD binary classification onset vs control-local confusion matrices', 25, 30, labels=[binary_classifier_OvsCL_l1.classes_, binary_classifier_OvsCL_l2.classes_], fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_confusion_matrix_OvsCL', '.svg'))

In [ ]:
df_predicted_labels_OC = create_dataframe(['true labels', 'predict_labels_l1', 'predict_labels_l2'], [ybin_OC_test, ybinary_prediction_OC_l1, ybinary_prediction_OC_l2])
df_predicted_labels_OL = create_dataframe(['true labels', 'predict_labels_l1', 'predict_labels_l2'], [ybin_OL_test, ybinary_prediction_OL_l1, ybinary_prediction_OL_l2])
df_predicted_labels_CL = create_dataframe(['true labels', 'predict_labels_l1', 'predict_labels_l2'], [ybin_CL_test, ybinary_prediction_CL_l1, ybinary_prediction_CL_l2])
df_predicted_labels_OvsCL = create_dataframe(['true labels', 'predict_labels_l1', 'predict_labels_l2'], [ybin_OvsCL_test, ybinary_prediction_OvsCL_l1, ybinary_prediction_OvsCL_l2])

In [ ]:
save_data(df_predicted_labels_OC, path=generate_new_file_path(imported_matrix, '_dataframe_SGD_predicted_labels_OC' + ''))
save_data(df_predicted_labels_OL, path=generate_new_file_path(imported_matrix, '_dataframe_SGD_predicted_labels_OL' + ''))
save_data(df_predicted_labels_CL, path=generate_new_file_path(imported_matrix, '_dataframe_SGD_predicted_labels_CL' + ''))
save_data(df_predicted_labels_OvsCL, path=generate_new_file_path(imported_matrix, '_dataframe_SGD_predicted_labels_OvsCL' + ''))

In [ ]:
matrix_for_dict = multi_to_single_header(matrix_tobe_classified, get_value_from_json(matrix_json, 'header_num_dict'))

Coefficients

In [ ]:
coef_binary_OC_l1 = relevant_coefficients(binary_classifier_OC_l1)
dic_coef_binary_OC_l1 = create_dict_coefficients(matrix_for_dict, coef_binary_OC_l1, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_OC_l1 = remove_zero_coefficients(dic_coef_binary_OC_l1)

coef_binary_OC_l2 = relevant_coefficients(binary_classifier_OC_l2)
dic_coef_binary_OC_l2 = create_dict_coefficients(matrix_for_dict, coef_binary_OC_l2, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_OC_l2 = remove_zero_coefficients(dic_coef_binary_OC_l2)

coef_binary_OL_l1 = relevant_coefficients(binary_classifier_OL_l1)
dic_coef_binary_OL_l1 = create_dict_coefficients(matrix_for_dict, coef_binary_OL_l1, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_OL_l1 = remove_zero_coefficients(dic_coef_binary_OL_l1)

coef_binary_OL_l2 = relevant_coefficients(binary_classifier_OL_l2)
dic_coef_binary_OL_l2 = create_dict_coefficients(matrix_for_dict, coef_binary_OL_l2, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_OL_l2 = remove_zero_coefficients(dic_coef_binary_OL_l2)

coef_binary_CL_l1 = relevant_coefficients(binary_classifier_CL_l1)
dic_coef_binary_CL_l1 = create_dict_coefficients(matrix_for_dict, coef_binary_CL_l1, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_CL_l1 = remove_zero_coefficients(dic_coef_binary_CL_l1)

coef_binary_CL_l2 = relevant_coefficients(binary_classifier_CL_l2)
dic_coef_binary_CL_l2 = create_dict_coefficients(matrix_for_dict, coef_binary_CL_l2, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_CL_l2 = remove_zero_coefficients(dic_coef_binary_CL_l2)

coef_binary_OvsCL_l1 = relevant_coefficients(binary_classifier_OvsCL_l1)
dic_coef_binary_OvsCL_l1 = create_dict_coefficients(matrix_for_dict, coef_binary_OvsCL_l1, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_OvsCL_l1 = remove_zero_coefficients(dic_coef_binary_OvsCL_l1)

coef_binary_OvsCL_l2 = relevant_coefficients(binary_classifier_OvsCL_l2)
dic_coef_binary_OvsCL_l2 = create_dict_coefficients(matrix_for_dict, coef_binary_OvsCL_l2, get_value_from_json(matrix_json, 'dic_column_name'))
dic_coef_binary_filtered_OvsCL_l2 = remove_zero_coefficients(dic_coef_binary_OvsCL_l2)

In [ ]:
### RETRIEVE OLD RESULTS

df_coef_OC_binary_l1_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OC_l1_recovery'), sep='\t')
df_coef_OC_binary_l2_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OC_l2_recovery'), sep='\t')
df_coef_OL_binary_l1_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OL_l1_recovery'), sep='\t')
df_coef_OL_binary_l2_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OL_l2_recovery'), sep='\t')
df_coef_CL_binary_l1_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_CL_l1_recovery'), sep='\t')
df_coef_CL_binary_l2_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_CL_l2_recovery'), sep='\t')
df_coef_OvsCL_binary_l1_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OvsCL_l1_recovery'), sep='\t')
df_coef_OvsCL_binary_l2_filtered_recovery = pd.read_csv(generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OvsCL_l2_recovery'), sep='\t')

recovered_OC_l1_keys = df_coef_OC_binary_l1_filtered_recovery['Metabolite ID'].tolist()
recovered_OC_l2_keys = df_coef_OC_binary_l2_filtered_recovery['Metabolite ID'].tolist()
recovered_OL_l1_keys = df_coef_OL_binary_l1_filtered_recovery['Metabolite ID'].tolist()
recovered_OL_l2_keys = df_coef_OL_binary_l2_filtered_recovery['Metabolite ID'].tolist()
recovered_CL_l1_keys = df_coef_CL_binary_l1_filtered_recovery['Metabolite ID'].tolist()
recovered_CL_l2_keys = df_coef_CL_binary_l2_filtered_recovery['Metabolite ID'].tolist()
recovered_OvsCL_l1_keys = df_coef_OvsCL_binary_l1_filtered_recovery['Metabolite ID'].tolist()
recovered_OvsCL_l2_keys = df_coef_OvsCL_binary_l2_filtered_recovery['Metabolite ID'].tolist()

recovered_OC_l1_labels = df_coef_OC_binary_l1_filtered_recovery['Non-zero coefficients'].tolist()
recovered_OC_l2_labels = df_coef_OC_binary_l2_filtered_recovery['Non-zero coefficients'].tolist()
recovered_OL_l1_labels = df_coef_OL_binary_l1_filtered_recovery['Non-zero coefficients'].tolist()
recovered_OL_l2_labels = df_coef_OL_binary_l2_filtered_recovery['Non-zero coefficients'].tolist()
recovered_CL_l1_labels = df_coef_CL_binary_l1_filtered_recovery['Non-zero coefficients'].tolist()
recovered_CL_l2_labels = df_coef_CL_binary_l2_filtered_recovery['Non-zero coefficients'].tolist()
recovered_OvsCL_l1_labels = df_coef_OvsCL_binary_l1_filtered_recovery['Non-zero coefficients'].tolist()
recovered_OvsCL_l2_labels = df_coef_OvsCL_binary_l2_filtered_recovery['Non-zero coefficients'].tolist()

assert len(recovered_OC_l1_keys) == len(recovered_OC_l1_labels)
assert len(recovered_OC_l2_keys) == len(recovered_OC_l2_labels)
assert len(recovered_OL_l1_keys) == len(recovered_OL_l1_labels)
assert len(recovered_OL_l2_keys) == len(recovered_OL_l2_labels)
assert len(recovered_CL_l1_keys) == len(recovered_CL_l1_labels)
assert len(recovered_CL_l2_keys) == len(recovered_CL_l2_labels)
assert len(recovered_OvsCL_l1_keys) == len(recovered_OvsCL_l1_labels)
assert len(recovered_OvsCL_l2_keys) == len(recovered_OvsCL_l2_labels)

dic_coef_binary_filtered_OC_l1 = {}
dic_coef_binary_filtered_OC_l2 = {}
dic_coef_binary_filtered_OL_l1 = {}
dic_coef_binary_filtered_OL_l2 = {}
dic_coef_binary_filtered_CL_l1 = {}
dic_coef_binary_filtered_CL_l2 = {}
dic_coef_binary_filtered_OvsCL_l1 = {}
dic_coef_binary_filtered_OvsCL_l2 = {}

for i in range(len(recovered_OC_l1_keys)):
	dic_coef_binary_filtered_OC_l1[str(recovered_OC_l1_keys[i])] = recovered_OC_l1_labels[i]
for i in range(len(recovered_OC_l2_keys)):
	dic_coef_binary_filtered_OC_l2[str(recovered_OC_l2_keys[i])] = recovered_OC_l2_labels[i]
for i in range(len(recovered_OL_l1_keys)):
	dic_coef_binary_filtered_OL_l1[str(recovered_OL_l1_keys[i])] = recovered_OL_l1_labels[i]
for i in range(len(recovered_OL_l2_keys)):
	dic_coef_binary_filtered_OL_l2[str(recovered_OL_l2_keys[i])] = recovered_OL_l2_labels[i]
for i in range(len(recovered_CL_l1_keys)):
	dic_coef_binary_filtered_CL_l1[str(recovered_CL_l1_keys[i])] = recovered_CL_l1_labels[i]
for i in range(len(recovered_CL_l2_keys)):
	dic_coef_binary_filtered_CL_l2[str(recovered_CL_l2_keys[i])] = recovered_CL_l2_labels[i]
for i in range(len(recovered_OvsCL_l1_keys)):
	dic_coef_binary_filtered_OvsCL_l1[str(recovered_OvsCL_l1_keys[i])] = recovered_OvsCL_l1_labels[i]
for i in range(len(recovered_OvsCL_l2_keys)):
	dic_coef_binary_filtered_OvsCL_l2[str(recovered_OvsCL_l2_keys[i])] = recovered_OvsCL_l2_labels[i]

print("Data recovered")

In [ ]:
x_label = 'Metabolite ID'
y_label = '50 biggest coefficients'

df_coef_OC_binary_l1_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OC_l1.keys(), dic_coef_binary_filtered_OC_l1.values()])
df_coef_OC_binary_l1_filtered[y_label] = df_coef_OC_binary_l1_filtered[y_label].abs()
df_coef_OC_binary_l1_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_OC_binary_l1_filtered = df_coef_OC_binary_l1_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification onset vs control - SGD l1 penalty", df_coef_OC_binary_l1_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_plot_coef_binary_OC_l1', '.svg'))

df_coef_OC_binary_l2_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OC_l2.keys(), dic_coef_binary_filtered_OC_l2.values()])
df_coef_OC_binary_l2_filtered[y_label] = df_coef_OC_binary_l2_filtered[y_label].abs()
df_coef_OC_binary_l2_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_OC_binary_l2_filtered = df_coef_OC_binary_l2_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification onset vs control - SGD l2 penalty", df_coef_OC_binary_l2_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_plot_coef_binary_OC_l2', '.svg'))

df_coef_OL_binary_l1_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OL_l1.keys(), dic_coef_binary_filtered_OL_l1.values()])
df_coef_OL_binary_l1_filtered[y_label] = df_coef_OL_binary_l1_filtered[y_label].abs()
df_coef_OL_binary_l1_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_OL_binary_l1_filtered = df_coef_OL_binary_l1_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification onset vs local - SGD l1 penalty", df_coef_OL_binary_l1_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_plot_coef_binary_OL_l1', '.svg'))

df_coef_OL_binary_l2_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OL_l2.keys(), dic_coef_binary_filtered_OL_l2.values()])
df_coef_OL_binary_l2_filtered[y_label] = df_coef_OL_binary_l2_filtered[y_label].abs()
df_coef_OL_binary_l2_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_OL_binary_l2_filtered = df_coef_OL_binary_l2_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification onset vs local - SGD l2 penalty", df_coef_OL_binary_l2_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_plot_coef_binary_OL_l2', '.svg'))

df_coef_CL_binary_l1_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_CL_l1.keys(), dic_coef_binary_filtered_CL_l1.values()])
df_coef_CL_binary_l1_filtered[y_label] = df_coef_CL_binary_l1_filtered[y_label].abs()
df_coef_CL_binary_l1_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_CL_binary_l1_filtered = df_coef_CL_binary_l1_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification control vs local - SGD l1 penalty", df_coef_CL_binary_l1_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_plot_coef_binary_CL_l1', '.svg'))

df_coef_CL_binary_l2_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_CL_l2.keys(), dic_coef_binary_filtered_CL_l2.values()])
df_coef_CL_binary_l2_filtered[y_label] = df_coef_CL_binary_l2_filtered[y_label].abs()
df_coef_CL_binary_l2_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_CL_binary_l2_filtered = df_coef_CL_binary_l2_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification control vs local - SGD l2 penalty", df_coef_CL_binary_l2_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix), get_file_name(imported_matrix), '_SGD_plot_coef_binary_CL_l2', '.svg'))

df_coef_OvsCL_binary_l1_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OvsCL_l1.keys(), dic_coef_binary_filtered_OvsCL_l1.values()])
df_coef_OvsCL_binary_l1_filtered[y_label] = df_coef_OvsCL_binary_l1_filtered[y_label].abs()
df_coef_OvsCL_binary_l1_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_OvsCL_binary_l1_filtered = df_coef_OvsCL_binary_l1_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification onset vs control-local - SGD l1 penalty", df_coef_OvsCL_binary_l1_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix),get_file_name(imported_matrix), '_SGD_plot_coef_binary_OvsCL_l1', '.svg'))

df_coef_OvsCL_binary_l2_filtered = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OvsCL_l2.keys(), dic_coef_binary_filtered_OvsCL_l2.values()])
df_coef_OvsCL_binary_l2_filtered[y_label] = df_coef_OvsCL_binary_l2_filtered[y_label].abs()
df_coef_OvsCL_binary_l2_filtered.sort_values(by=y_label, ascending=False, inplace=True)
df_coef_OvsCL_binary_l2_filtered = df_coef_OvsCL_binary_l2_filtered[:50 :]
plot_features_importances(get_file_name(imported_matrix) + " - Binary classification onset vs control-local - SGD l2 penalty", df_coef_OvsCL_binary_l2_filtered, x_label, y_label, FI_XSIZE, FI_YSIZE, fig_save=FIGSAVE, fig_path=merge_path_filename(get_file_path(imported_matrix),get_file_name(imported_matrix), '_SGD_plot_coef_binary_OvsCL_l2', '.svg'))

Obtain features name

In [ ]:
name_features_OC_l1 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_OC_l1.keys()), 'Alignment.ID', 'Metabolite.name')
name_features_OC_l2 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_OC_l2.keys()), 'Alignment.ID', 'Metabolite.name')

name_features_OL_l1 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_OL_l1.keys()), 'Alignment.ID', 'Metabolite.name')
name_features_OL_l2 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_OL_l2.keys()), 'Alignment.ID', 'Metabolite.name')

name_features_CL_l1 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_CL_l1.keys()), 'Alignment.ID', 'Metabolite.name')
name_features_CL_l2 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_CL_l2.keys()), 'Alignment.ID', 'Metabolite.name')

name_features_OvsCL_l1 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_OvsCL_l1.keys()), 'Alignment.ID', 'Metabolite.name')
name_features_OvsCL_l2 = get_metabolites_name_byID(matrix_for_dict, list(dic_coef_binary_filtered_OvsCL_l2.keys()), 'Alignment.ID', 'Metabolite.name')

In [ ]:
df_all_features_OC_l1 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OC_l1.keys(), dic_coef_binary_filtered_OC_l1.values(), name_features_OC_l1])
df_all_features_OC_l2 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OC_l2.keys(), dic_coef_binary_filtered_OC_l2.values(), name_features_OC_l2])

df_all_features_OL_l1 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OL_l1.keys(), dic_coef_binary_filtered_OL_l1.values(), name_features_OL_l1])
df_all_features_OL_l2 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OL_l2.keys(), dic_coef_binary_filtered_OL_l2.values(), name_features_OL_l2])

df_all_features_CL_l1 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_CL_l1.keys(), dic_coef_binary_filtered_CL_l1.values(), name_features_CL_l1])
df_all_features_CL_l2 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_CL_l2.keys(), dic_coef_binary_filtered_CL_l2.values(), name_features_CL_l2])

df_all_features_OvsCL_l1 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OvsCL_l1.keys(), dic_coef_binary_filtered_OvsCL_l1.values(), name_features_OvsCL_l1])
df_all_features_OvsCL_l2 = create_dataframe([x_label, y_label], [dic_coef_binary_filtered_OvsCL_l2.keys(), dic_coef_binary_filtered_OvsCL_l2.values(), name_features_OvsCL_l2])

In [ ]:
save_data(df_all_features_OC_l1, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OC_l1'))
save_data(df_all_features_OC_l2, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OC_l2'))

save_data(df_all_features_OL_l1, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OL_l1'))
save_data(df_all_features_OL_l2, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OL_l2'))

save_data(df_all_features_CL_l1, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_CL_l1'))
save_data(df_all_features_CL_l2, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_CL_l2'))

save_data(df_all_features_OvsCL_l1, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OvsCL_l1'))
save_data(df_all_features_OvsCL_l2, path=generate_new_file_path(imported_matrix, '_dataframe_coefficients_binary_OvsCL_l2'))

In [ ]:
imported_matrix.close()
matrix_json.close()